In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix

In [ ]:
# Загрузка данных
data = pd.read_csv("skab_valve_1.csv", sep=';', parse_dates=['datetime'])
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')

In [ ]:
# Отображение первых строк данных
print(data.head())

In [ ]:
# Поиск аномалий с использованием IQR
def detect_anomalies_iqr(data, column):
    # Вычисление первого (Q1) и третьего (Q3) квартилей
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    # Вычисление интерквартильного размаха (IQR)
    IQR = Q3 - Q1
    # Определение границ для аномалий
    lower_bound_1_5 = Q1 - (1.5 * IQR)
    upper_bound_1_5 = Q3 + (1.5 * IQR)
    lower_bound_3 = Q1 - (3 * IQR)
    upper_bound_3 = Q3 + (3 * IQR)

    return lower_bound_1_5, upper_bound_1_5, lower_bound_3, upper_bound_3

In [ ]:
# Аномалии для термопары
column = 'Thermocouple'
lower_bound_1_5, upper_bound_1_5, lower_bound_3, upper_bound_3 = detect_anomalies_iqr(data, column)

In [ ]:
# Обозначение аномалий в данных
data['anomaly_iqr_1_5'] = ((data[column] < lower_bound_1_5) | (data[column] > upper_bound_1_5)).astype(int)
data['anomaly_iqr_3'] = ((data[column] < lower_bound_3) | (data[column] > upper_bound_3)).astype(int)

In [ ]:
# Визуализация данных с аномалиями
plt.figure(figsize=(14, 7))
plt.plot(data['datetime'], data['Thermocouple'], label='Температура термопары')
plt.scatter(data['datetime'][data['anomaly_iqr_1_5'] == 1],
            data['Thermocouple'][data['anomaly_iqr_1_5'] == 1],
            color='red', label='Аномалии (1.5 IQR)')
plt.scatter(data['datetime'][data['anomaly_iqr_3'] == 1],
            data['Thermocouple'][data['anomaly_iqr_3'] == 1],
            color='orange', label='Аномалии (3 IQR)')
plt.title("Выявление аномалий в данных термопары")
plt.xlabel("Дата и время")
plt.ylabel("Температура термопары")
plt.legend()
plt.show()

In [ ]:
# Confusion matrix для IQR
conf_matrix_iqr_1_5 = confusion_matrix(data['anomaly'], data['anomaly_iqr_1_5'])
conf_matrix_iqr_3 = confusion_matrix(data['anomaly'], data['anomaly_iqr_3'])
print("Матрица ошибок (1.5 IQR):\n", conf_matrix_iqr_1_5)
print("Матрица ошибок (3 IQR):\n", conf_matrix_iqr_3)

In [ ]:
# Создание новых признаков для Feature Engineering
data['hour'] = data['datetime'].dt.hour  # Час
data['day_of_week'] = data['datetime'].dt.dayofweek  # День недели
data['month'] = data['datetime'].dt.month  # Месяц
data['rolling_mean'] = data['Thermocouple'].rolling(window=5).mean()  # Скользящее среднее
data['rolling_std'] = data['Thermocouple'].rolling(window=5).std()  # Скользящее стандартное отклонение

In [ ]:
# Кластеризация с использованием DBSCAN
features = data[['Accelerometer1RMS', 'Accelerometer2RMS', 'Current', 'Pressure',
                 'Temperature', 'Thermocouple', 'Voltage', 'Volume Flow RateRMS',
                 'hour', 'day_of_week', 'month', 'rolling_mean', 'rolling_std']].fillna(0)

In [ ]:
# Фильтрация аномалий на основе кластеризации
data['anomaly_dbscan'] = (data['dbscan_labels'] == -1).astype(int)

In [ ]:
# Визуализация результатов DBSCAN
plt.figure(figsize=(14, 7))
plt.scatter(data['datetime'], data['Thermocouple'], c=data['dbscan_labels'], cmap='coolwarm', label='Кластеры DBSCAN')
plt.title("Кластеризация данных термопары с помощью DBSCAN")
plt.xlabel("Дата и время")
plt.ylabel("Температура термопары")
plt.colorbar(label='Метка кластера')
plt.show()

In [ ]:
conf_matrix_dbscan = confusion_matrix(data['anomaly'], data['anomaly_dbscan'])
print("Матрица ошибок (DBSCAN):\n", conf_matrix_dbscan)

In [ ]:
iso_forest = IsolationForest(contamination=0.1)
data['iso_anomaly'] = iso_forest.fit_predict(features)
data['iso_anomaly'] = data['iso_anomaly'].map({1: 0, -1: 1})  # Преобразуем -1 и 1 в 1 и 0

In [ ]:
# Визуализация результатов Isolation Forest
plt.figure(figsize=(14, 7))
plt.plot(data['datetime'], data['Thermocouple'], label='Данные термопары')
plt.scatter(data['datetime'][data['iso_anomaly'] == 1],
            data['Thermocouple'][data['iso_anomaly'] == 1],
            color='green', label='Аномалии Isolation Forest')
plt.title("Выявление аномалий с помощью Isolation Forest")
plt.xlabel("Дата и время")
plt.ylabel("Температура термопары")
plt.legend()
plt.show()

In [ ]:
# Confusion matrix для Isolation Forest
conf_matrix_iso = confusion_matrix(data['anomaly'], data['iso_anomaly'])
print("Матрица ошибок (Isolation Forest):\n", conf_matrix_iso)